# Building blocks in Haystack: components and pipelines

In the [previous notebook](data_classes.ipynb), we learned how we can store structured and unstructured data through Documents objects, as well as dataframe, ByteStream, ChatMessage and StreamingChunk objects. We also learned how to store these objects into a Document Store. In this notebook, we will explore how to store and retrieve data from a Haystack Document store. Let's take a look at its architecture.

Haystack's architecture leverages components as its core elements, each performing specific functions like text processing or summarization. These components are designed to be connected into pipelines, which orchestrate the flow of data and manage task execution in a structured manner. The Pipeline class facilitates this by allowing the addition and connection of components, which must have unique input and output points for data transfer.

Pipelines are the backbone of NLP applications in Haystack, functioning as directed graphs where nodes are components and edges dictate data flow. They ensure smooth data processing, handle errors, and support debugging through visualization tools that help developers trace and optimize the data journey.

Haystack emphasizes modularity and flexibility, providing a range of pre-built components while also supporting custom ones for specific needs. The framework's pipelines enable the assembly of sophisticated NLP applications, integrating various functionalities into a cohesive system. In this notebook we will explore key components. In  the pipelines.ipynb notebook we will see how to connect them into pipelines.

### Introduction to components

Within Haystack, we can find the following key ready-made components. There are more, but for now we will focus on these as we get started with Haystack's functionality.

![](./images/haystack-components.png)

### Embedding components

In this example, `docs` is a list of `Document` objects with text content to be embedded. The `OpenAIDocumentEmbedder` is initialized with an OpenAI API key and is used to generate embeddings for each document. The embeddings are then printed out for each document in the `docs` list.

#### OpenAIDocumentEmbedder

In [ ]:
from haystack.preview import Document
from haystack.preview.components.embedders import OpenAIDocumentEmbedder
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv("./../../.env")
api_key = os.getenv("OPENAI_API_KEY")

# List of documents to embed
docs = [Document(content="The quick brown fox jumps over the lazy dog."), 
        Document(content="To be or not to be, that is the question.")]

# Initialize the embedder with your OpenAI API key
document_embedder = OpenAIDocumentEmbedder(api_key=api_key)
""
# Run the embedder to get embeddings
result = document_embedder.run(docs)

# Access the embeddings stored in the documents
for doc in result['documents']:
    print(doc.embedding[0:2])

Taking a look at the result data structure

In [ ]:
result

The metadata shows the model and usage.

In [ ]:
result['metadata']

#### OpenAITextEmbedder

In this snippet, `text_embedder` is created with an OpenAI API key and used to generate an embedding for the string "I love pizza!". The resulting embedding and associated metadata are then printed out.

In [ ]:
from haystack.preview.components.embedders import OpenAITextEmbedder

# Initialize the text embedder with your OpenAI API key
text_embedder = OpenAITextEmbedder(api_key=api_key)

# Text you want to embed
text_to_embed = "I love pizza!"

# Embed the text and print the result
result_text= text_embedder.run(text_to_embed)

# Output will include the embedding vector and metadata
# {'embedding': [0.017020374536514282, -0.023255806416273117, ...],
#  'metadata': {'model': 'text-embedding-ada-002',
#               'usage': {'prompt_tokens': 4, 'total_tokens': 4}}}


In [ ]:
result_text.keys()

As before, we can access the embeddings through the embedding key

In [ ]:
result_text['embedding'][0:2]

In [ ]:
result_text['metadata']